In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook contains the code and steps for merging the reviews dataset with the processed calendar dataset.

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)

pandas version: 1.5.3
numpy version: 1.22.4


In [4]:
# data path
data_path = '[insert path to csv file here]'

In [5]:
# Load datasets and turn "date" column as datetime object
df_rev = pd.read_csv(data_path + "processed_reviews.csv", parse_dates=['date'])
df_cal = pd.read_csv(data_path + "processed_calendar.csv", parse_dates=['date'])

# Combine processed reviews data with calendar data

In [7]:
# Extract month, year, and day of the week into separate columns for reviews df
df_rev['month'] = df_rev['date'].dt.month
df_rev['year'] = df_rev['date'].dt.year
df_rev['day_of_week'] = df_rev['date'].dt.day_name()

In [8]:
# Extract month, year, and day of the week into separate columns for calendar df
df_cal['month'] = df_cal['date'].dt.month
df_cal['year'] = df_cal['date'].dt.year
df_cal['day_of_week'] = df_cal['date'].dt.day_name()

In [68]:
# Merge the DataFrames based on the four similar columns
merged_df = pd.merge(df_rev, df_cal, on=['month', 'year', 'day_of_week', 'listing_id'], how='inner')


In [69]:
# Remove date_x column (this is the date column from reviews dataset)
rev_cal_df = merged_df.drop('date_x', axis=1)

In [70]:
# Rename the column from 'date_y' to 'date' (this is the calendar 'date' column)
rev_cal_df = rev_cal_df.rename(columns={'date_y': 'date'})

In [71]:
# Convert "day_of_week" column to categorical
rev_cal_df['day_of_week'] = rev_cal_df['day_of_week'].astype('category')

# Convert "year" column to categorical
rev_cal_df['year'] = rev_cal_df['year'].astype('category')

# Convert "month" column to categorical
rev_cal_df['month'] = rev_cal_df['month'].astype('category')



In [72]:
# make a copy of merged review/calendar data
revcal_copy = rev_cal_df.copy()

The dataframe 'final_df' will be used to merge into the listings data.

# Combine the new review_calendar dataframe with the Listings data

In [48]:
# Load in datasets
listings_df = pd.read_csv(data_path + 'processed_listings.csv')

In [73]:
# rename 'listing_id" to "id" to match the column from listings_df
revcal_copy = revcal_copy.rename(columns= {'listing_id' : 'id'})

In [74]:
# inner join, only the matching rows between the two dataframes
# are included in the merged result, excluding any rows that don't have a match
final_df = revcal_copy.merge(listings_df, on='id', how='inner')


In [75]:
# drop new added column 'Unnamed: 0'
final_df = final_df.drop('Unnamed: 0', axis=1)


In [78]:
# print total amount of null value
print("Before:", final_df.isnull().sum().sum())

# Drop rows with null values
final_df = final_df.dropna()

# print total amount of null value
print("After:", final_df.isnull().sum().sum())

Before: 25
After: 0


In [ ]:
# save dataframe to system
# final_df.to_csv(final_df.csv', index=False)

# Merge calendar and listing dataframes
This dataset will not include any information from the reviews dataset

In [80]:
# rename 'listing_id" to "id" to match the column from listings_df
df_cal = df_cal.rename(columns= {'listing_id' : 'id'})

In [81]:
# inner join, only the matching rows between the two dataframes
# are included in the merged result, excluding any rows that don't have a match
listing_cal_final = df_cal.merge(listings_df, on='id', how='inner')


In [82]:
# check for missing values
listing_cal_final.isnull().sum().sum()

0

In [ ]:
# drop new added column 'Unnamed: 0'
listing_cal_final = listing_cal_final.drop('Unnamed: 0', axis=1)


In [84]:
print(listing_cal_final.shape)
listing_cal_final.head(5)

(18105646, 91)


,id,date,price,month,year,day_of_week,host_id,host_is_superhost,host_listings_count,host_total_listings_count,...,mex_arte,mckinney,umlauf,bonnell,museum_weird,review_scores_location,room_type_Hotel room,room_type_Private room,room_type_Shared room,semi_interpolated_reviews
0,5456,2022-06-09,95.0,6,2022,Thursday,8028,1.0,2.0,4.0,...,1.068293,7.51835,3.078306,7.736014,0.842357,4.73,0,0,0,0
1,5456,2022-06-10,150.0,6,2022,Friday,8028,1.0,2.0,4.0,...,1.068293,7.51835,3.078306,7.736014,0.842357,4.73,0,0,0,0
2,5456,2022-06-11,150.0,6,2022,Saturday,8028,1.0,2.0,4.0,...,1.068293,7.51835,3.078306,7.736014,0.842357,4.73,0,0,0,0
3,5456,2022-06-12,95.0,6,2022,Sunday,8028,1.0,2.0,4.0,...,1.068293,7.51835,3.078306,7.736014,0.842357,4.73,0,0,0,0
4,5456,2022-06-13,95.0,6,2022,Monday,8028,1.0,2.0,4.0,...,1.068293,7.51835,3.078306,7.736014,0.842357,4.73,0,0,0,0


In [89]:
# Convert columns to a categorical variable
categorical_features = ['day_of_week','year', 'id', 'host_id', 'neighbourhood_cleansed']
listing_cal_final[categorical_features] = listing_cal_final[categorical_features].astype('category')

In [ ]:
# save to csv file
# listing_cal_final.to_csv(data_path + 'df_no_reviews.csv', index=False)